In [2]:
import numpy as np
# from numpy import zeros, ones, eye
import matplotlib.pyplot as plt
# import tensorflow as tf

In [3]:
X = np.array([[0,0,1],\
              [0,1,1],\
              [1,0,1],\
              [1,1,1]], dtype=np.float32)#1 appended in the last column is the bias

t = { #dictionary for getting both the target logic values and the correlated string 
    "AND": np.array([0, 0, 0, 1], dtype=np.float32),
    "NAND": np.array([1, 1, 1, 0], dtype=np.float32),
    "OR": np.array([0, 1, 1, 1], dtype=np.float32),
    "NOR": np.array([1, 0, 0, 0], dtype=np.float32),
    "XOR": np.array([0, 1, 1, 0], dtype=np.float32) 
}

### function definitions
y_pred, online_perceptron are correct. Not sure about cost.
The nested for loop is terrible, I know. It was a required part of the implementation

In [4]:
def cost(y,x,w):
    J = y*x*w
    print(y*x*w)
    return J

def y_pred(X, w_pckg, gate_name, print_result):
    # w_pckg[0]: w vector
    # w_pckg[1]: no. of iterations to converge
    # w_pckg[2]: max no. of iterations
    # w_pckg[3]: converged or not (1 or 0)
    # w_pckg[4]: learning rate
    
    y = np.heaviside(X @ w_pckg[0], 0) # 0 or 1 for output
    if print_result:
        print(gate_name + ": " + str(y))
        if w_pckg[3]:
            print(str(w_pckg[1]) + " iterations to convergence")
        else:
            print("did not converge")
        print(str(w_pckg[2]) + " (max iterations)")
        print(str(w_pckg[4]) + " (learning rate)")
    return y

def online_perceptron(X, t, rho, epochs):
    y = np.array([0, 0, 0, 0], dtype=np.float32)
    w_k = np.array([0, 0, 0], dtype=np.float32)
    converged = 0
    for i in range(epochs):            
        for j in range(len(X)):
             #zeros in [w_k, 0,...] is so that function selects the right w vector
            y[j] = y_pred(X[j,:], [w_k, 0, 0, 0], '', 0) 
            err = t[j] - y[j]
            w_k = w_k + rho*err*X[j,:]
            
        #it stops once all are matching
        if len(np.where(t == y)[0]) == 4:
            converged = 1
            break
    return [w_k, i+1, epochs, converged, rho]

In [5]:
#runs for loops for all logic gate within the dictionary t
# for one learning rate AND a one no. of epochs
def predict_percept_all_gates(X, t, rho, epochs):
    w_pkg = {}
    for i in t:
        w_pkg[i] = online_perceptron(X, t[i], rho, epochs)
    
    y_p = {}
    k = 0

    for i in t: #
        y_p[i] = y_pred(X, w_pkg[i], i, 1)
        print()
        k += 1
        
    return [y_p, w_pkg]

In [7]:
y = {}

# y: dictionary; to access members, use an element of the list RHO
# e.g. y[RHO[0]] == [y_pred, w_pkg], a list with a dict y_pred AND a dict the w_pkg 
#                                  y_pred: dict with 5 elements (indeces are the logic gates' names, capital letters)
#                                          each one a logic gate's prediction
#                                  w_pkg: dict with 5 elements (same indexing as y_pred)
#                                          each one a list with the the following elements:
#                                               [w_k, no_of_iterations_to_convergence, epochs, converged, rho]
#                                               [3-element np array(w vect), scalar, scalar, scalar, scalar]

#learning reates
RHO = np.array([.0001,.001,.01,.01,.1,1], dtype=np.float32)
#epochs
iterations = [200, 2_000, 20_000, 200_000, 2_000_000]

for i in range(len(RHO)):
    print("===== learning rate: " + str(RHO[i]) + " =============================")
    y[RHO[i]] = predict_percept_all_gates(X, t, RHO[i], iterations[0])

===== learning rate: 1e-04 =============================
AND: [0. 0. 0. 1.]
6 iterations to convergence
200 (max iterations)
1e-04 (learning rate)

NAND: [1. 1. 1. 0.]
6 iterations to convergence
200 (max iterations)
1e-04 (learning rate)

OR: [0. 1. 1. 1.]
4 iterations to convergence
200 (max iterations)
1e-04 (learning rate)

NOR: [1. 0. 0. 0.]
4 iterations to convergence
200 (max iterations)
1e-04 (learning rate)

XOR: [1. 1. 0. 0.]
did not converge
200 (max iterations)
1e-04 (learning rate)

===== learning rate: 0.001 =============================
AND: [0. 0. 0. 1.]
6 iterations to convergence
200 (max iterations)
0.001 (learning rate)

NAND: [1. 1. 1. 0.]
6 iterations to convergence
200 (max iterations)
0.001 (learning rate)

OR: [0. 1. 1. 1.]
4 iterations to convergence
200 (max iterations)
0.001 (learning rate)

NOR: [1. 0. 0. 0.]
4 iterations to convergence
200 (max iterations)
0.001 (learning rate)

XOR: [1. 1. 0. 0.]
did not converge
200 (max iterations)
0.001 (learning rate)

In [6]:
# example on first rho, y_pred selected, XOR
y[RHO[0]][0]["NAND"]

array([1., 1., 1., 0.], dtype=float32)